# NEURAL NETWORK :

## Input , Hidden , Output Layers :

In [163]:
import numpy as np

In [164]:
# make class instance for input layer :
import math
class dense_layer:
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01*np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
        
    def forward_prop(self, inputs):
        self.input = inputs 
        self.output = np.dot(inputs, self.weights) + self.biases
        return self.output
        
    def backward_prop(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        
        # Update Parameter ;
        self.weights -= learning_rate * weights_error
        self.biases -= learning_rate * output_error
        return input_error

## Activation Function :

### Sigmoid :

In [165]:
def sigmoid(inputs):
    return 1/(1 + np.exp(-inputs))
def sigmoid_der(inputs):
    return 1/(1 + np.exp(-inputs))*(1-1/(1 + np.exp(-inputs)))

# Loss :

In [166]:
class loss:
    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        data_loss = np.mean(sample_losses)
        
class loss_categoricalcrossentropy(loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        y_pred_clapped = np.clip(y_pred, 1e-7, 1-1e-7)
        
        if len(y_true.shape) == 1:
            correct_confidence = y_pred_clapped[range(samples), y_true]
            
        if len(y_true.shape) == 2:
            correct_confidence = np.sum(y_pred_clapped*y_true, axis=1)
            
        negative_log_likelihoods = -np.log(correct_confidence)
        return negative_log_likelihoods

# Neural Stucture :

## Base Layer :

In [167]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None
    
    def forward_prop(self, input):
        raise NotImplementedError
    def backward_prop(self, output_error, learning_rate):
        raise NotImplementedError

# Dense Layer :

In [168]:
class Dense(Layer):
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01*np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
    def forward_prop(self, inputs):
        self.input = inputs 
        self.output = np.dot(inputs, self.weights) + self.biases
        return self.output
    def backward_prop(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        
        # Update Parameter ;
        self.weights -= learning_rate * weights_error
        self.biases -= learning_rate * output_error
        return input_error

# Activation Layer :

In [169]:
class ActivationLayer(Layer):
    def __init__(self, activation, activation_der):
        self.activation = activation
        self.activation_der = activation_der

    # returns the activated input
    def forward_prop(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    # Returns input_error=dE/dX for a given output_error=dE/dY.
    # learning_rate is not used because there is no "learnable" parameters.
    def backward_prop(self, output_error, learning_rate):
        return self.activation_der(self.input) * output_error

# Error Calculation :

In [170]:
import numpy as np

# loss function and its derivative
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;

# Final Neural Network :

In [171]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use(self, loss, loss_der):
        self.loss = loss
        self.loss_der = loss_der

    # predict output for given input
    def predict(self, input_data):
        # sample dimension first
        samples = len(input_data)
        result = []

        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_prop(output)
            result.append(output)

        return result

    # train the network
    def fit(self, x_train, y_train, epochs, learning_rate):
        # sample dimension first
        samples = len(x_train)

        # training loop
        for i in range(epochs):
            err = 0
            for j in range(samples):
                # forward propagation
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_prop(output)

                # compute loss (for display purpose only)
                err += self.loss(y_train[j], output)

                # backward propagation
                error = self.loss_der(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_prop(error, learning_rate)

            # calculate average error on all samples
            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))

# Build Model :

In [172]:
from nnfs.datasets import spiral_data
# x_train, y_train = spiral_data(samples=100, classes=3 )
# training data
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])

# # network
model = Network()
model.add(Dense(2, 3))
model.add(ActivationLayer(sigmoid, sigmoid_der))
model.add(Dense(3, 1))
model.add(ActivationLayer(sigmoid, sigmoid_der))

In [173]:
x_train.shape

(4, 1, 2)

In [174]:
y_train.shape

(4, 1, 1)

# Train Model :

In [175]:
model.use(mse, mse_prime)

In [176]:
model.fit(x_train, y_train, epochs=1000, learning_rate=0.1)

epoch 1/1000   error=0.255524
epoch 2/1000   error=0.255523
epoch 3/1000   error=0.255522
epoch 4/1000   error=0.255521
epoch 5/1000   error=0.255521
epoch 6/1000   error=0.255520
epoch 7/1000   error=0.255519
epoch 8/1000   error=0.255519
epoch 9/1000   error=0.255518
epoch 10/1000   error=0.255517
epoch 11/1000   error=0.255516
epoch 12/1000   error=0.255516
epoch 13/1000   error=0.255515
epoch 14/1000   error=0.255514
epoch 15/1000   error=0.255514
epoch 16/1000   error=0.255513
epoch 17/1000   error=0.255512
epoch 18/1000   error=0.255512
epoch 19/1000   error=0.255511
epoch 20/1000   error=0.255510
epoch 21/1000   error=0.255510
epoch 22/1000   error=0.255509
epoch 23/1000   error=0.255508
epoch 24/1000   error=0.255508
epoch 25/1000   error=0.255507
epoch 26/1000   error=0.255506
epoch 27/1000   error=0.255506
epoch 28/1000   error=0.255505
epoch 29/1000   error=0.255504
epoch 30/1000   error=0.255504
epoch 31/1000   error=0.255503
epoch 32/1000   error=0.255502
epoch 33/1000   e

epoch 310/1000   error=0.255325
epoch 311/1000   error=0.255324
epoch 312/1000   error=0.255323
epoch 313/1000   error=0.255323
epoch 314/1000   error=0.255322
epoch 315/1000   error=0.255322
epoch 316/1000   error=0.255321
epoch 317/1000   error=0.255320
epoch 318/1000   error=0.255320
epoch 319/1000   error=0.255319
epoch 320/1000   error=0.255319
epoch 321/1000   error=0.255318
epoch 322/1000   error=0.255317
epoch 323/1000   error=0.255317
epoch 324/1000   error=0.255316
epoch 325/1000   error=0.255316
epoch 326/1000   error=0.255315
epoch 327/1000   error=0.255314
epoch 328/1000   error=0.255314
epoch 329/1000   error=0.255313
epoch 330/1000   error=0.255312
epoch 331/1000   error=0.255312
epoch 332/1000   error=0.255311
epoch 333/1000   error=0.255311
epoch 334/1000   error=0.255310
epoch 335/1000   error=0.255309
epoch 336/1000   error=0.255309
epoch 337/1000   error=0.255308
epoch 338/1000   error=0.255308
epoch 339/1000   error=0.255307
epoch 340/1000   error=0.255306
epoch 34

epoch 641/1000   error=0.255132
epoch 642/1000   error=0.255132
epoch 643/1000   error=0.255131
epoch 644/1000   error=0.255131
epoch 645/1000   error=0.255130
epoch 646/1000   error=0.255130
epoch 647/1000   error=0.255129
epoch 648/1000   error=0.255128
epoch 649/1000   error=0.255128
epoch 650/1000   error=0.255127
epoch 651/1000   error=0.255127
epoch 652/1000   error=0.255126
epoch 653/1000   error=0.255126
epoch 654/1000   error=0.255125
epoch 655/1000   error=0.255125
epoch 656/1000   error=0.255124
epoch 657/1000   error=0.255124
epoch 658/1000   error=0.255123
epoch 659/1000   error=0.255122
epoch 660/1000   error=0.255122
epoch 661/1000   error=0.255121
epoch 662/1000   error=0.255121
epoch 663/1000   error=0.255120
epoch 664/1000   error=0.255120
epoch 665/1000   error=0.255119
epoch 666/1000   error=0.255119
epoch 667/1000   error=0.255118
epoch 668/1000   error=0.255117
epoch 669/1000   error=0.255117
epoch 670/1000   error=0.255116
epoch 671/1000   error=0.255116
epoch 67

epoch 997/1000   error=0.254948
epoch 998/1000   error=0.254947
epoch 999/1000   error=0.254947
epoch 1000/1000   error=0.254947


## Predict:

In [177]:
pred = model.predict(x_train)
pred

[array([[0.50013069]]),
 array([[0.50012917]]),
 array([[0.50012944]]),
 array([[0.5001278]])]